In [1]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle as cPickle  # Sorry, workaround for Python 3.4
import gzip

# Third-party libraries
import numpy as np

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')  # Make it work with Py3.x
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [17]:
%time training_data, validation_data, test_data = load_data_wrapper()

CPU times: user 1.68 s, sys: 536 ms, total: 2.22 s
Wall time: 2.23 s


In [3]:
%time x, y = zip(*training_data)

x = np.array(x)
x = x.reshape(50000, 784)

y = np.array(y)
y = y.reshape(50000, 10)


CPU times: user 61.7 ms, sys: 44.9 ms, total: 107 ms
Wall time: 110 ms


In [4]:
import keras


import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils

from sklearn.preprocessing import StandardScaler

np.random.seed(1337) # for reproducibility


def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler



Using Theano backend.


/Users/tjl/Development/sandpit/lca16/source_packages/Theano/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [5]:
 
nb_classes = y.shape[1]
print(nb_classes, 'classes')

dims = x.shape[1]
print(dims, 'dims')

 


10 classes
784 dims


In [6]:

print("Building model...")

LAYER_ONE_SIZE = 1024
ITERATIONS = 15

model = Sequential()
model.add(Dense(output_dim=LAYER_ONE_SIZE, input_dim=dims, init='glorot_uniform'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Dense(LAYER_ONE_SIZE, LAYER_ONE_SIZE, init='glorot_uniform'))
# model.add(PReLU((LAYER_ONE_SIZE,)))
# model.add(BatchNormalization((LAYER_ONE_SIZE,)))
# model.add(Dropout(0.5))

# model.add(Dense(LAYER_ONE_SIZE, LAYER_ONE_SIZE, init='glorot_uniform'))
# model.add(PReLU((LAYER_ONE_SIZE,)))
# model.add(BatchNormalization((LAYER_ONE_SIZE,)))
# model.add(Dropout(0.5))

model.add(Dense(output_dim=nb_classes, input_dim=LAYER_ONE_SIZE, init='glorot_uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam")

print("Training model...")

%time model.fit(x, y, nb_epoch=ITERATIONS, batch_size=128, validation_split=0.15)  

Building model...
Training model...
Train on 42500 samples, validate on 7500 samples
Epoch 1/15
42500/42500 [==============================] - 7s - loss: 0.5229 - val_loss: 0.2689
Epoch 2/15
42500/42500 [==============================] - 7s - loss: 0.2152 - val_loss: 0.1900
Epoch 3/15
42500/42500 [==============================] - 7s - loss: 0.1485 - val_loss: 0.1473
Epoch 4/15
42500/42500 [==============================] - 6s - loss: 0.1120 - val_loss: 0.1275
Epoch 5/15
42500/42500 [==============================] - 6s - loss: 0.0910 - val_loss: 0.1130
Epoch 6/15
42500/42500 [==============================] - 6s - loss: 0.0746 - val_loss: 0.1097
Epoch 7/15
42500/42500 [==============================] - 6s - loss: 0.0649 - val_loss: 0.1095
Epoch 8/15
42500/42500 [==============================] - 6s - loss: 0.0579 - val_loss: 0.1077
Epoch 9/15
42500/42500 [==============================] - 6s - loss: 0.0517 - val_loss: 0.1007
Epoch 10/15
42500/42500 [==============================] - 7

In [18]:
p_x, p_y = zip(*test_data)

p_x = np.array(p_x)
p_x = p_x.reshape(10000, 784)

p_y = np.array(p_y)
p_y = p_y.reshape(10000)

In [19]:
%time preds = model.predict(p_x)
print(preds[1])
print(np.argmax(preds[1]))

CPU times: user 559 ms, sys: 14 ms, total: 573 ms
Wall time: 254 ms
[  6.49067422e-09   1.31841416e-05   9.99986768e-01   2.02232986e-08
   2.97270437e-11   5.72542447e-12   9.15234832e-11   4.14213559e-11
   2.25691313e-10   1.00165949e-14]
2


In [20]:
pred_idx = [np.argmax(a) for a in preds]

In [21]:
pairs = zip(pred_idx, p_y)
print(len(p_y))

10000


In [22]:
number_correct = sum([int(a == b) for a, b in pairs])
print(number_correct)

9808


In [23]:
number_incorrect = len(p_y) - number_correct
print(number_incorrect)

192


In [24]:
eps = 0.000001  # avoid division by zero
success_rate = number_correct / float((number_correct + number_incorrect + eps))
print(success_rate)

0.98079999990192
